# Lakh Pianoroll Dataset 5 Preparation

## Installations

install
1. muspy & pypianoroll via pip install in environment folder (e.g. /Users/kai/anaconda3/opt/envs/MusiCAN/bin)
2. fluidsynth via conda install -c conda-forge fluidsynth

In [ ]:
from IPython.display import clear_output, Audio, display
from ipywidgets import interact, IntSlider

import os
import os.path
import random
import datetime
import json

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from tqdm import tqdm # valuebar for iterations

In [ ]:
import muspy
import pypianoroll

In [ ]:
muspy.download_musescore_soundfont() # if you didn't do it already 

In [ ]:
muspy.download_bravura_font() # if you didn't do it already 

## Prepare Genre Labels

In [ ]:
# create labels & id_list

genre_list = ['Rap', 'Latin', 'International', 'Electronic', 
              'Country', 'Folk', 'Blues', 'Reggae', 'Jazz',
              'Vocal', 'New-Age', 'RnB', 'Pop_Rock'] # genre <-> numeric label = index

id_list = [] # id = MillionSongsDataset ID
track_label_list = []
for path in os.listdir("unprepared_data/id_lists_amg"):
    filepath = "unprepared_data/id_lists_amg/" + path
    
    with open(filepath) as f:
        ids = [line.rstrip() for line in f]
        number_of_ids = len(ids)
        id_list.extend(ids)
    
    genre_no = genre_list.index(path[8:-4])
    track_label_list.extend([genre_no] * number_of_ids)  

In [ ]:
# make sure no multiple genre label 
n = 0
id_array = np.array(id_list)
for id_1 in id_list:
    n_ids = np.sum(id_array == id_1)
    n += (n_ids - 1)
n == 0

## Prepare Piano-rolls

In [ ]:
# helper function

def msd_id_to_dirs(msd_id):
    """Given an MillionSongsDataset ID, generate the path prefix.
    E.g. TRABCD12345678 -> A/B/C/TRABCD12345678"""
    return(msd_id[2] + "/" + msd_id[3] + "/" + msd_id[4] + "/" + msd_id)

In [ ]:
# set values
dataset_root = "unprepared_data/lpd_5/lpd_5_cleansed/"

n_pitches = 6*12  # number of pitches
lowest_pitch = 2*12  # MIDI note number of the lowest pitch
beat_resolution = 4 # temporal resolution of a beat (in timestep), 24 in data, 12 for MusiGAN
bars_per_instance = 12 # number of bars per instance in prepared data 
min_n_notes = 8 # minimal number of notes per instance

# for later..
bar_resolution = 4 * beat_resolution
sample_size = 4 * bars_per_instance # number of beats per instance created by track-cropping, 4 bars for MusiGAN

In [ ]:
# helper function

def plot_pianoroll(pr):
    """
    pr: Pitches x Time
    """
    
    fig = plt.figure(figsize = (15, 6))
    plt.imshow(pr.T)
    plt.show()

In [ ]:
# iterate over all the songs in the ID list & prepare data

data_list = []
data_label_list = []

for i, msd_id in enumerate(tqdm(id_list)):
    
    # load multitrack as a pypianoroll.Multitrack instance
    song_dir = dataset_root + msd_id_to_dirs(msd_id)
    filename = os.listdir(song_dir)[0]
    multitrack = pypianoroll.load(song_dir + "/" + filename)

    # downsample
    multitrack = multitrack.set_resolution(beat_resolution, rounding = "round")
    
    # select piano trck (2nd one)
    piano_track = multitrack.tracks[1]
    
    # pad into mupliples of bars such that later piano can be splitted into bars easily
    piano_track = piano_track.pad_to_multiple(bar_resolution)
    
    # array conversion (shape: time x pitches)
    pianoroll = piano_track.pianoroll
    
    # binarize pianoroll
    pianoroll = (pianoroll > 0)
    
    # fix pitch range
    pianoroll = pianoroll[:, lowest_pitch : lowest_pitch + n_pitches] # (shape: time x pitches))

    # slice into pieces of sequences without empty bars
    reshaped = pianoroll.reshape(-1, bar_resolution, n_pitches)
    reshaped = pianoroll.reshape(-1, bar_resolution * n_pitches)
    empty_bars_mask = np.any(reshaped, axis = 1)
    split_after_this_bar_mask = (np.diff(empty_bars_mask) != 0)
    split_after_this_bar_numbers = np.arange(len(empty_bars_mask) - 1)[split_after_this_bar_mask]
    split_indices = ((split_after_this_bar_numbers + 1) * bar_resolution) 
    split_list = np.split(pianoroll, split_indices)
    
    # crop each split into smaller training samples
    
    start = np.any(split_list[0]) # = 0 if first split empty, else 1
    for split in split_list[1 - start::2]: # only select non-empty slices
        n_timesteps = bars_per_instance * bar_resolution # time steps per instance
        if split.shape[0] >= n_timesteps: # crop only what is at least as big as one crop should be
            split = split[ : split.shape[0] - (split.shape[0] % n_timesteps), :] # make sure: number of total timesteps of track % n_timesteps == 0, else skip last bar
            splits = split.reshape((-1, n_timesteps, n_pitches)) # crops x time x pitches
            
            # append splits & labels
            data_list.append(splits)
            data_label_list.extend([track_label_list[i]] * splits.shape[0]) # append as many labels as crops added
    
data_array = np.concatenate(data_list, axis = 0) # (shape: n_instances x n_timesteps x n_pitches)
label_array = np.array(data_label_list)
print(f"Successfully collect {len(data_array)} samples from {len(id_list)} songs")
print(f"Data shape : {data_array.shape}, {label_array.shape}")

In [ ]:
for i in range(20):
    plot_pianoroll(data_array[i])

In [ ]:
# save prepared data

## create unique file directory to save data
timestamp = datetime.datetime.now()
file_directory = f"./prepared_data/lpd5_{timestamp}"
os.makedirs(file_directory)
os.makedirs(file_directory + "/audio_examples") # for later..

## save preparation parameters as json file
prep_pars_dict = {"n_pitches": n_pitches,
                 "lowest_pitch": lowest_pitch,
                 "beat_resolution": beat_resolution, 
                  "beats_per_instance": sample_size}
with open(file_directory + "/preparation_params.json", "w") as file:
    json.dump(prep_pars_dict, file, indent = 6)

## save data as compressed npz files
np.savez_compressed(file_directory + "/prepared_arrays.npz", data=data_array, labels=label_array)

## Evaluate Prepared Data

In [ ]:
# load data

file_directory = f"./prepared_data/lpd5_full_12bars"  # change if wanted
loaded_data = np.load(file_directory + "/prepared_arrays.npz")
loaded_data_array, loaded_label_array = loaded_data["data"], loaded_data["labels"]

In [ ]:
# number of data for each genre?

plt.hist(loaded_label_array)
plt.ylabel("# of instances")
plt.xlabel("genre label")

print(genre_list)

In [ ]:
# plot total pitch range 

n_pitches_array = loaded_data_array.sum(axis = 1).sum(axis = 0)
plt.plot(np.arange(72), n_pitches_array, "ro")
plt.ylabel("# of pitches")
plt.xlabel("pitches")
# plt.yscale("log")

In [ ]:
# convert random instances of loaded data to wave (audio) file & display them

n = 10 # number or random examples
rand_idxs = np.random.randint(0, len(loaded_label_array), n)

for i in tqdm(rand_idxs):
    X, y = loaded_data_array[i, :, :], loaded_label_array[i]
    
    genre_of_X = genre_list[y]
    
    X_padded = np.pad(X, ((0, 0), (lowest_pitch, 128 - lowest_pitch - n_pitches))) # complete pitch range
    X_music = muspy.from_pianoroll_representation(X_padded > 0, 
                resolution = beat_resolution, encode_velocity = False) # convert to muspy.music_object

    X_timestamp = datetime.datetime.now()
    muspy.write_audio(path = file_directory + f"/audio_examples/{genre_of_X}_{X_timestamp}.wav", 
                      music = X_music) 
    
    # display audio & show pianoroll
    print(genre_of_X + ":")
    display(Audio(filename = file_directory + f"/audio_examples/{genre_of_X}_{X_timestamp}.wav"))
    muspy.visualization.show_pianoroll(X_music)
    plt.show()

In [ ]:
# convert into pytorch dataset & dataloader
batch_size = 15

# convert to pytorch tensor
data_tensor = torch.as_tensor(loaded_data_array, dtype=torch.float32)
label_tensor = torch.as_tensor(loaded_label_array, dtype=torch.int)

# create pytorch dataset & dataloader
dataset = torch.utils.data.TensorDataset(data_tensor, label_tensor)
lpd5_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [ ]:
liste = [1,2,3][0]

In [ ]:
np.array([0.,  1.,  2.])[np.array([True, True, False])]